# DrugBank GraphSynergy Toxicity Score Analysis #

- GraphSynergy paper can be found here: https://academic.oup.com/jamia/article/28/11/2336/6362567?login=true#305111898

In [1]:
# Import everything needed
from matplotlib.patches import Patch
from scipy import stats
from sklearn.metrics import r2_score
from statsmodels.stats.multitest import multipletests
from preprocessing_functions import *
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scikit_posthocs as sp
import seaborn as sns

GraphSynergy creates toxicity score embeddings by:
1. Aggregating the all neighbors within 2 degrees of drugs' two_hop proteins on a PPIN
2. Creating a similarity score by computing the inner product of drug embeddings

So let's examine each of these options in how well toxicity category distributions are (Kruskal Wallis) and if there is a relationship in increasing order with the toxicity categories (Jonckheere Terpestra Test)

In [2]:
STRING_G = get_STRING_graph()

# Get all the drug combinations
drug_combos_df = pd.read_csv('data_processed/drugbank_processed_combos_syntoxtargallpw_string.csv')
targets_df = pd.read_csv('data_processed/drugbank_syntoxtarg_allpw_string.csv')

# Create a dataframe of drug combinations and their targets as well as the targets' neighbors within 2 hops
drug_combos_within_2_hops = {}
for index, row in drug_combos_df.iterrows():
    drugA = row['drug_row']
    drugB = row['drug_col']
    key = (drugA, drugB)
    if key not in drug_combos_within_2_hops:
        A_targets = set(targets_df[targets_df['drug_name'] == drugA]['STRING_ID'].dropna().values)
        B_targets = set(targets_df[targets_df['drug_name'] == drugB]['STRING_ID'].dropna().values)
        A_neighbors = set()
        B_neighbors = set()
        for target in A_targets:
            if target in STRING_G.nodes:
                A_neighbors.update(STRING_G.neighbors(target))
        for target in B_targets:
            if target in STRING_G.nodes:
                B_neighbors.update(STRING_G.neighbors(target))
        A_2nd_neighbors = set()
        B_2nd_neighbors = set()
        for neighbor in A_neighbors:
            if neighbor in STRING_G.nodes:
                A_2nd_neighbors.update(STRING_G.neighbors(neighbor))
        for neighbor in B_neighbors:
            if neighbor in STRING_G.nodes:
                B_2nd_neighbors.update(STRING_G.neighbors(neighbor))
        
        drug_combos_within_2_hops[(drugA, drugB)] = {
            'A_targets': A_targets,
            'B_targets': B_targets,
            'A_neighbors': A_neighbors,
            'B_neighbors': B_neighbors,
            'A_2nd_neighbors': A_2nd_neighbors,
            'B_2nd_neighbors': B_2nd_neighbors
        } 


Original shape of STRING edge list, physical detailed: (1477610, 6)


In [3]:
# For each drug combination, compute the jaccard similarity between the sets of neighborhood proteins of drug A and drug B
for index, row in drug_combos_df.iterrows():
    key = (row['drug_row'], row['drug_col'])
    # Get the full set of proteins within 2 hops of drug A and drug B
    A_full_within_2_hops = drug_combos_within_2_hops[key]['A_targets'].union(drug_combos_within_2_hops[key]['A_neighbors']).union(drug_combos_within_2_hops[key]['A_2nd_neighbors'])
    B_full_within_2_hops = drug_combos_within_2_hops[key]['B_targets'].union(drug_combos_within_2_hops[key]['B_neighbors']).union(drug_combos_within_2_hops[key]['B_2nd_neighbors'])
    # Compute the Jaccard similarity
    if len(A_full_within_2_hops.union(B_full_within_2_hops)) == 0:
        drug_combos_df.at[index, 'two_hop_jaccard'] = -1
        print('No targets found for drug combination', key)
        continue
    
    two_hop_jaccard = jaccard_similarity(A_full_within_2_hops, B_full_within_2_hops)
    drug_combos_df.at[index, 'two_hop_jaccard'] = two_hop_jaccard

In [4]:
# Test if jaccard similarity distribution is normal
print("Is the two hop JS distribution normal? Normal test p-value: ", stats.normaltest(drug_combos_df['two_hop_jaccard']))

# Look at histogram of two hop jaccard similarities
plt.hist(drug_combos_df['two_hop_jaccard'], bins=20)
plt.xlabel('2-Hop Neighbors Jaccard Similarity', fontsize=20)
plt.ylabel('Frequency')
plt.title('Histogram of T2-Hop Neighbors Jaccard Similarity')
plt.savefig('results/two_hop_analysis/twohopneighborjs_hist_drugbank.png')
plt.close()

# Save description of two hop jaccard similarities
drug_combos_df['two_hop_jaccard'].describe().to_csv('results/two_hop_analysis/twohopneighborjs_stats_drugbank.csv')

Is the two hop JS distribution normal? Normal test p-value:  NormaltestResult(statistic=7500.229694446114, pvalue=0.0)


- Kruskal Wallis Test
- Dunn Posthoc Test with Bonferroni Correction
- Jonckheere Terpestra Test
- ANOVA
- T Test with Bonferroni Correction

In [5]:
# Run Kruskal Wallis test on jaccard similarities with targeting Major, Moderate, and Minor toxicity

# two_hop jaccard
major_twohop = drug_combos_df[drug_combos_df['toxicity_category'] == 'Major']['two_hop_jaccard'].dropna()
moderate_twohop = drug_combos_df[drug_combos_df['toxicity_category'] == 'Moderate']['two_hop_jaccard'].dropna()
minor_twohop = drug_combos_df[drug_combos_df['toxicity_category'] == 'Minor']['two_hop_jaccard'].dropna()
print(f'Major two_hop jaccard: {len(major_twohop)}')
print(f'Moderate two_hop jaccard: {len(moderate_twohop)}')
print(f'Minor two_hop jaccard: {len(minor_twohop)}')

h_statistic_twohop_tox, p_value_twohop_tox = stats.kruskal(major_twohop, moderate_twohop, minor_twohop)
print(f'Kruskal-Wallis H statistic for two_hop jaccard: {h_statistic_twohop_tox}')
print(f'P-value for two_hop jaccard: {p_value_twohop_tox}')

# Filter drug_combos_df for only Major, Moderate, and Minor toxicity categories and remove pairs with two_hop jaccard similarity of None
drug_combos_df_tox_twohop = drug_combos_df[drug_combos_df['toxicity_category'].isin(['Major', 'Moderate', 'Minor'])]
drug_combos_df_tox_twohop = drug_combos_df[~drug_combos_df_tox_twohop['two_hop_jaccard'].isnull()]
dunn_twohop_tox = sp.posthoc_dunn(drug_combos_df_tox_twohop, val_col='two_hop_jaccard', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for two_hop jaccard:' + str(dunn_twohop_tox))

# jonkcheere terpestra test -- is there a trend in the overlap (jaccard similarity) as you increase toxicity from minor to major?
jaccard_major_samples = major_twohop.values.tolist()
jaccard_moderate_samples = moderate_twohop.values.tolist()
jaccard_minor_samples = minor_twohop.values.tolist()
jt_incr_twohop = jonckheere_terpestra_test([jaccard_minor_samples, jaccard_moderate_samples, jaccard_major_samples])
print("Increasing toxicity for two_hop overlap: ", jt_incr_twohop )
jt_decr_twohop = jonckheere_terpestra_test([jaccard_major_samples, jaccard_moderate_samples, jaccard_minor_samples])
print("Decreasing toxicity for two_hop overlap: ", jt_decr_twohop)

# Run an ANOVA on two_hop jaccard similarity for Major, Moderate, and Minor toxicity categories
f_statistic_anova_twohop_tox, p_value_anova_twohop_tox = stats.f_oneway(major_twohop, moderate_twohop, minor_twohop)
print(f'ANOVA F-statistic for two_hop jaccard similarity: {f_statistic_anova_twohop_tox}')
print(f'P-value for two_hop jaccard similarity: {p_value_anova_twohop_tox}')

# Run a T test on two_hop jaccard similarity between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_twohop = stats.ttest_ind(major_twohop, minor_twohop)
major_moderate_twohop = stats.ttest_ind(major_twohop, moderate_twohop)
moderate_minor_twohop = stats.ttest_ind(moderate_twohop, minor_twohop)

# Run bonferroni correction on the p-values
ttest_p_values = [major_minor_twohop[1], major_moderate_twohop[1], moderate_minor_twohop[1]]
ttest_p_values_corrected = multipletests(ttest_p_values, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_twohop[1]} Corrected: {ttest_p_values_corrected[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_twohop[1]} Corrected: {ttest_p_values_corrected[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_twohop[1]} Corrected: {ttest_p_values_corrected[1][2]}')


with open('results/two_hop_analysis/two_hop_targ_jaccard_drugbank.tsv', 'w') as f:
    f.write('Level\tTest\tTest statistic\tP-value\n')
    f.write('Two-Hop\tKruskal-Wallis\t{:.4e}\t{:.4e}\n'.format(h_statistic_twohop_tox, p_value_twohop_tox))
    f.write('Two-Hop\tDunn post-hoc\tMajor/Minor\t{:.4e}\n'.format(dunn_twohop_tox.iloc[0,1]))
    f.write('Two-Hop\tDunn post-hoc\tMajor/Moderate\t{:.4e}\n'.format(dunn_twohop_tox.iloc[0, 2]))
    f.write('Two-Hop\tDunn post-hoc\tModerate/Minor\t{:.4e}\n'.format(dunn_twohop_tox.iloc[1, 2]))
    f.write('Two-Hop\tJonckheere-Terpstra Increasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_incr_twohop[0], jt_incr_twohop[1]))
    f.write('Two-Hop\tJonckheere-Terpstra Decreasing toxicity\t{:.4e}\t{:.4e}\n'.format(jt_decr_twohop[0], jt_decr_twohop[1]))
    f.write('Two-Hop\tANOVA\t{:.4e}\t{:.4e}\n'.format(f_statistic_anova_twohop_tox, p_value_anova_twohop_tox))
    f.write('Two-Hop\tT-test (bonf cor)\tMajor/Minor\t{:.4e}\n'.format(ttest_p_values_corrected[1][0]))
    f.write('Two-Hop\tT-test (bonf cor)\tMajor/Moderate\t{:.4e}\n'.format(ttest_p_values_corrected[1][1]))
    f.write('Two-Hop\tT-test (bonf cor)\tModerate/Minor\t{:.4e}\n'.format(ttest_p_values_corrected[1][2]))

Major two_hop jaccard: 36864
Moderate two_hop jaccard: 19074
Minor two_hop jaccard: 6790
Kruskal-Wallis H statistic for two_hop jaccard: 6153.156295433598
P-value for two_hop jaccard: 0.0
Dunn post-hoc test for two_hop jaccard:          Major         Minor      Moderate
Major       1.0  0.000000e+00  0.000000e+00
Minor       0.0  1.000000e+00  1.584148e-99
Moderate    0.0  1.584148e-99  1.000000e+00
Increasing toxicity for two_hop overlap:  (77.6376878530536, 0.0)
Decreasing toxicity for two_hop overlap:  (-77.6376878530536, 1.0)
ANOVA F-statistic for two_hop jaccard similarity: 5369.102136978421
P-value for two_hop jaccard similarity: 0.0
Major/Minor T-test: 0.0 Corrected: 0.0
Major/Moderate T-test: 0.0 Corrected: 0.0
Moderate/Minor T-test: 4.659757477204994e-63 Corrected: 1.3979272431614983e-62


- Violin Plot

In [6]:
colors = ['#20965D', '#FFBC42', '#D81159']
tox_order = ['Minor', 'Moderate', 'Major']
color_dict = dict(zip(tox_order, colors))
legend_elements = [
    Patch(facecolor=color_dict[cat], label = cat) for cat in tox_order
]

ax = sns.violinplot(data=drug_combos_df, x='toxicity_category', y='two_hop_jaccard', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set(xlabel='', ylabel='Neighbor Jaccard Similarity')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/two_hop_analysis/twohop_js_v_toxcatwohop_drugbank_violin.png', dpi=700)
plt.close()

- Strip Plot

In [7]:
ax = sns.stripplot(data=drug_combos_df, x='toxicity_category', y='two_hop_jaccard', palette=color_dict, hue='toxicity_category', order=tox_order)
sns.boxplot( # plot the mean line
    showmeans=True,
    meanline=True,
    meanprops={'color': 'k', 'ls': '-', 'lw': 1},
    medianprops={'visible': False},
    whiskerprops={'visible': False},
    zorder=10,
    x="toxicity_category",
    y="two_hop_jaccard",
    data=drug_combos_df,
    showfliers=False,
    showbox=False,
    showcaps=False,
    ax=ax
)
ax.set(xlabel='', ylabel='Neighbor Jaccard Similarity')
plt.xticks(fontsize=20)
ax.yaxis.label.set_size(20)
plt.tight_layout()
plt.savefig('results/two_hop_analysis/twohop_js_v_toxcatwohop_drugbank_strip.png', dpi=700)
plt.close()

- Correlation Scatter Plots (Neighboring Proteins withing 2 Hops Jaccard Similarity v Synergy Scores)
- R^2 value
- Best fit line
- Pearson correlation coefficient
- Spearman correlation coefficient

In [8]:
x_twohop = drug_combos_df['two_hop_jaccard']
y_bliss_true = drug_combos_df['synergy_bliss']
y_loewe_true = drug_combos_df['synergy_loewe']
y_hsa_true = drug_combos_df['synergy_hsa']
y_zip_true = drug_combos_df['synergy_zip']
y_smax_true = drug_combos_df['S_max']
y_smean_true = drug_combos_df['S_mean']
y_ssum_true = drug_combos_df['S_sum']

####### BLISS ########
# Let's plot two hop jaccard similarity against bliss synergy scores
plt.scatter(x_twohop, y_bliss_true)

# Best fit line
z_twohop_bliss = np.polyfit(x_twohop, y_bliss_true, 1)
p_twohop_bliss = np.poly1d(z_twohop_bliss)
y_bliss_pred = p_twohop_bliss(x_twohop)
r_squared_twohop_bliss = r2_score(y_bliss_true, y_bliss_pred)
plt.plot(x_twohop, y_bliss_pred, "r-", alpha=0.8, label=f'R² = {r_squared_twohop_bliss:.3f}')
plt.xlabel('Neighborhood Jaccard Similarity', fontsize=20)
plt.ylabel('Bliss Synergy Score', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('results/two_hop_analysis/bliss_v_twohop_drugbank_scatter.png', dpi=700)
plt.close()

# Calculate Pearson/Spearman correlation coefficient between two hop jaccard similarity and bliss synergy score
two_hop_bliss_corr = x_twohop.corr(y_bliss_true)
two_hop_bliss_spearman_corr = x_twohop.corr(y_bliss_true, method='spearman')

####### HSA ########
# Let's plot two hop jaccard similarity against hsa synergy scores
plt.scatter(x_twohop, y_hsa_true)

# Best fit line
z_twohop_hsa = np.polyfit(x_twohop, y_hsa_true, 1)
p_twohop_hsa = np.poly1d(z_twohop_hsa)
y_hsa_pred = p_twohop_hsa(x_twohop)
r_squared_twohop_hsa = r2_score(y_hsa_true, y_hsa_pred)
plt.plot(x_twohop, y_hsa_pred, "r-", alpha=0.8, label=f'R² = {r_squared_twohop_hsa:.3f}')
plt.xlabel('Neighborhood Jaccard Similarity', fontsize=20)
plt.ylabel('HSA Synergy Score', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('results/two_hop_analysis/hsa_v_twohop_drugbank_scatter.png', dpi=700)
plt.close()

# Calculate Pearson/Spearman correlation coefficient between two hop jaccard similarity and hsa synergy score
two_hop_hsa_corr = x_twohop.corr(y_hsa_true)
two_hop_hsa_spearman_corr = x_twohop.corr(y_hsa_true, method='spearman')

####### LOEWE ########
# Let's plot two hop jaccard similarity against loewe synergy scores
plt.scatter(x_twohop, y_loewe_true)

# Best fit line
z_twohop_loewe = np.polyfit(x_twohop, y_loewe_true, 1)
p_twohop_loewe = np.poly1d(z_twohop_loewe)
y_loewe_pred = p_twohop_loewe(x_twohop)
r_squared_twohop_loewe = r2_score(y_loewe_true, y_loewe_pred)
plt.plot(x_twohop, y_loewe_pred, "r-", alpha=0.8, label=f'R² = {r_squared_twohop_loewe:.3f}')

plt.xlabel('Neighborhood Jaccard Similarity', fontsize=20)
plt.ylabel('Loewe Synergy Score', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('results/two_hop_analysis/loewe_v_twohop_drugbank_scatter.png', dpi=700)
plt.close()

# Calculate Pearson/Spearman correlation coefficient between two hop jaccard similarity and loewe synergy score
two_hop_loewe_corr = x_twohop.corr(y_loewe_true)
two_hop_loewe_spearman_corr = x_twohop.corr(y_loewe_true, method='spearman')

####### ZIP ########
# Let's plot two hop jaccard similarity against zip synergy scores
plt.scatter(x_twohop, y_zip_true)

# Best fit line
z_twohop_zip = np.polyfit(x_twohop, y_zip_true, 1)
p_twohop_zip = np.poly1d(z_twohop_zip)
y_zip_pred = p_twohop_zip(x_twohop)
r_squared_twohop_zip = r2_score(y_zip_true, y_zip_pred)
plt.plot(x_twohop, y_zip_pred, "r-", alpha=0.8, label=f'R² = {r_squared_twohop_zip:.3f}')

plt.xlabel('Neighborhood Jaccard Similarity', fontsize=20)
plt.ylabel('ZIP Synergy Score', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('results/two_hop_analysis/zip_v_twohop_drugbank_scatter.png', dpi=700)
plt.close()

# Calculate Pearson/Spearman correlation coefficient between two hop jaccard similarity and zip synergy score
two_hop_zip_corr = x_twohop.corr(y_zip_true)
two_hop_zip_spearman_corr = x_twohop.corr(y_zip_true, method='spearman')

###### S_MAX ########
# Let's plot two hop jaccard similarity against S_max synergy scores
plt.scatter(x_twohop, y_smax_true)

# Best fit line
z_twohop_smax = np.polyfit(x_twohop, y_smax_true, 1)
p_twohop_smax = np.poly1d(z_twohop_smax)
y_smax_pred = p_twohop_smax(x_twohop)
r_squared_twohop_smax = r2_score(y_smax_true, y_smax_pred)
plt.plot(x_twohop, y_smax_pred, "r-", alpha=0.8, label=f'R² = {r_squared_twohop_smax:.3f}')

plt.xlabel('Neighborhood Jaccard Similarity', fontsize=20)
plt.ylabel('S_max Synergy Score', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('results/two_hop_analysis/smax_v_twohop_drugbank_scatter.png', dpi=700)
plt.close()

# Calculate Pearson/Spearman correlation
two_hop_smax_corr = x_twohop.corr(y_smax_true)
two_hop_smax_spearman_corr = x_twohop.corr(y_smax_true, method='spearman')

###### S_MEAN ########
# Let's plot two hop jaccard similarity against S_mean synergy scores
plt.scatter(x_twohop, y_smean_true)

# Best fit line
z_twohop_smean = np.polyfit(x_twohop, y_smean_true, 1)
p_twohop_smean = np.poly1d(z_twohop_smean)
y_smean_pred = p_twohop_smean(x_twohop)
r_squared_twohop_smean = r2_score(y_smean_true, y_smean_pred)
plt.plot(x_twohop, y_smean_pred, "r-", alpha=0.8, label=f'R² = {r_squared_twohop_smean:.3f}')

plt.xlabel('Neighborhood Jaccard Similarity', fontsize=20)
plt.ylabel('S_mean Synergy Score', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('results/two_hop_analysis/smean_v_twohop_drugbank_scatter.png', dpi=700)
plt.close()

# Calculate Pearson/Spearman correlation
two_hop_smean_corr = x_twohop.corr(y_smean_true)
two_hop_smean_spearman_corr = x_twohop.corr(y_smean_true, method='spearman')

###### S_SUM ########
# Let's plot two hop jaccard similarity against S_sum synergy scores
plt.scatter(x_twohop, y_ssum_true)

# Best fit line
z_twohop_ssum = np.polyfit(x_twohop, y_ssum_true, 1)
p_twohop_ssum = np.poly1d(z_twohop_ssum)
y_ssum_pred = p_twohop_ssum(x_twohop)
r_squared_twohop_ssum = r2_score(y_ssum_true, y_ssum_pred)
plt.plot(x_twohop, y_ssum_pred, "r-", alpha=0.8, label=f'R² = {r_squared_twohop_ssum:.3f}')

plt.xlabel('Neighborhood Jaccard Similarity', fontsize=20)
plt.ylabel('S_sum Synergy Score', fontsize=20)
plt.legend()
plt.tight_layout()
plt.savefig('results/two_hop_analysis/ssum_v_twohop_drugbank_scatter.png', dpi=700)
plt.close()

# Calculate Pearson/Spearman correlation
two_hop_ssum_corr = x_twohop.corr(y_ssum_true)
two_hop_ssum_spearman_corr = x_twohop.corr(y_ssum_true, method='spearman')

# Write all the correlations and spearman correlations to a file
with open('results/two_hop_analysis/two_hop_v_synergy_correlations_drugbank.tsv', 'w') as f:
    f.write('Synergy score\tLevel\tPearson correlation coefficient\tSpearman correlation\tR squared value\n')
    f.write('Bliss\t2-Hop Neighborhood Jaccard Similarity\t{:.4e}\t{:.4e}\t{:.4e}\n'.format(two_hop_bliss_corr, two_hop_bliss_spearman_corr,r_squared_twohop_bliss))
    f.write('HSA\t2-Hop Neighborhood Jaccard Similarity\t{:.4e}\t{:.4e}\t{:.4e}\n'.format(two_hop_hsa_corr, two_hop_hsa_spearman_corr, r_squared_twohop_hsa))
    f.write('Loewe\t2-Hop Neighborhood Jaccard Similarity\t{:.4e}\t{:.4e}\t{:.4e}\n'.format(two_hop_loewe_corr, two_hop_loewe_spearman_corr, r_squared_twohop_loewe))
    f.write('ZIP\t2-Hop Neighborhood Jaccard Similarity\t{:.4e}\t{:.4e}\t{:.4e}\n'.format(two_hop_zip_corr, two_hop_zip_spearman_corr, r_squared_twohop_zip))
    f.write('S_max\t2-Hop Neighborhood Jaccard Similarity\t{:.4e}\t{:.4e}\t{:.4e}\n'.format(two_hop_smax_corr, two_hop_smax_spearman_corr, r_squared_twohop_smax))
    f.write('S_mean\t2-Hop Neighborhood Jaccard Similarity\t{:.4e}\t{:.4e}\t{:.4e}\n'.format(two_hop_smean_corr, two_hop_smean_spearman_corr, r_squared_twohop_smean))
    f.write('S_sum\t2-Hop Neighborhood Jaccard Similarity\t{:.4e}\t{:.4e}\t{:.4e}\n'.format(two_hop_ssum_corr, two_hop_ssum_spearman_corr, r_squared_twohop_ssum))